In [ ]:
!pip install imblearn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
from imblearn.over_sampling import SMOTENC
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTENC
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier

import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset

from scipy.optimize import linear_sum_assignment as linear_assignment

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f'{device} is available')

cuda:0 is available


## 대전 Cluster 17

In [ ]:
dae_17 = pd.read_csv('/content/drive/MyDrive/지역 치안 안전 데이터 분석/data/modeling data/daejeon_17.csv')
display(dae_17)

,date,accident_num,평균기온(°C),일강수량(mm),평균 풍속(m/s),평균 상대습도(%),평균 이슬점온도(°C),year,month,day,측정소명,O3,PM10,PM2.5,season,요일,workday
0,2020-01-01,0.0,-0.6,0.0,0.9,65.8,-6.4,2020,1,1,읍내동,0,0,0,winter,수요일,1
1,2020-01-02,1.0,1.3,0.0,0.9,76.9,-2.4,2020,1,2,읍내동,0,1,2,winter,목요일,0
2,2020-01-03,2.0,2.0,0.0,1.1,73.4,-2.5,2020,1,3,읍내동,0,1,2,winter,금요일,0
3,2020-01-04,0.0,0.6,0.0,0.9,78.1,-3.0,2020,1,4,읍내동,0,1,2,winter,토요일,1
4,2020-01-05,0.0,1.3,0.0,0.8,63.5,-5.7,2020,1,5,읍내동,0,1,1,winter,일요일,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1122,2023-01-27,0.0,-6.0,0.0,2.9,55.1,-14.2,2023,1,27,읍내동,0,1,1,winter,금요일,0
1123,2023-01-28,0.0,-5.8,0.0,1.4,60.4,-12.6,2023,1,28,읍내동,0,1,1,winter,토요일,1
1124,2023-01-29,0.0,-1.1,0.0,1.9,63.4,-7.7,2023,1,29,읍내동,0,1,1,winter,일요일,1
1125,2023-01-30,0.0,-0.4,0.0,1.7,42.8,-12.4,2023,1,30,읍내동,0,1,1,winter,월요일,0


In [ ]:
dae_17['date'] = pd.to_datetime(dae_17['date'])
dae_17['accident_hpn'] = dae_17['accident_num'].apply(lambda x: 1 if x > 0 else 0)
dae_17['accident_hpn'].value_counts()/dae_17['accident_hpn'].value_counts().sum()

0    0.606034
1    0.393966
Name: accident_hpn, dtype: float64

In [ ]:
X = dae_17.drop(['date','accident_num','측정소명','accident_hpn'], axis = 1)
y = dae_17['accident_hpn']

In [ ]:
print(X.shape)
print(y.shape)

(1127, 14)
(1127,)


In [ ]:
X_trn, X_tes, y_trn, y_tes = train_test_split(X, y, test_size = 0.2, stratify = y, random_state = 42)

print(X_trn.shape)
print(X_tes.shape)
print(y_trn.shape)
print(y_tes.shape)

(901, 14)
(226, 14)
(901,)
(226,)


### 데이터 비율이 비슷하면 oversampling 하지 말기

In [ ]:
smote = SMOTENC(random_state=0, categorical_features = list(range(5,14)))
X_res, y_res = smote.fit_resample(X_trn,y_trn)

In [ ]:
print(y_res.value_counts())
print(y_res.value_counts()/y_res.value_counts().sum())

1    612
0    612
Name: accident_hpn, dtype: int64
1    0.5
0    0.5
Name: accident_hpn, dtype: float64


In [ ]:
cat_features = ['year', 'month','day','season','요일']
num_features = ['평균기온(°C)','일강수량(mm)','평균 풍속(m/s)','평균 상대습도(%)','평균 이슬점온도(°C)','O3','PM10','PM2.5']

enc = OneHotEncoder(handle_unknown = 'ignore')
scaler = MinMaxScaler()

enc.fit(X_trn[cat_features])
scaler.fit(X_trn[num_features])

train_oh = enc.transform(X_trn[cat_features]).toarray()
test_oh = enc.transform(X_tes[cat_features]).toarray()

train_num = scaler.transform(X_trn[num_features])
test_num = scaler.transform(X_tes[num_features])

print(train_oh.shape)
print(test_oh.shape)


(901, 58)
(226, 58)


In [ ]:
X_trn = np.concatenate([train_num,train_oh,X_trn['workday'].values.reshape(-1,1)], axis = 1)
X_tes = np.concatenate([test_num,test_oh,X_tes['workday'].values.reshape(-1,1)], axis = 1)

print(X_trn.shape)
print(X_tes.shape)

(901, 67)
(226, 67)


In [ ]:
class TrnDataset(Dataset):

  def __init__(self,x_data,y_data):
    self.x_data = torch.FloatTensor(x_data)
    self.y_data = torch.LongTensor(y_data)
    self.len = len(self.x_data)

  def __getitem__(self,index):
    return self.x_data[index], self.y_data[index]

  def __len__(self):
    return self.len



In [ ]:
class TesDataset(Dataset):

  def __init__(self,x_data,y_data):
    self.x_data = torch.FloatTensor(x_data)
    self.y_data = torch.LongTensor(y_data)
    self.len = len(self.x_data)

  def __getitem__(self,index):
    return self.x_data[index], self.y_data[index]

  def __len__(self):
    return self.len



In [ ]:
train_data = TrnDataset(X_trn, y_trn.values)
test_data = TesDataset(X_tes, y_tes.values)

train_loader = DataLoader(train_data,batch_size=100,shuffle=True)
test_loader = DataLoader(test_data,batch_size=100,shuffle=True)

In [ ]:
arr, label = next(iter(train_loader))
print(arr.shape, label.shape)
arr2, label2 = next(iter(test_loader))
print(arr2.shape, label2.shape)

torch.Size([100, 67]) torch.Size([100])
torch.Size([100, 67]) torch.Size([100])


In [ ]:
latent_size = 4

In [ ]:
class Encoder(nn.Module):
  def __init__(self,latent_size):
    super(Encoder,self).__init__()

    self.encoder = nn.Sequential(
                      nn.Linear(67,32),
                      nn.ReLU(),
                      nn.Linear(32,16),
                      nn.ReLU(),
                      nn.Linear(16,latent_size),
                      nn.ReLU())

  def forward(self,x):
    return self.encoder(x)


class Decoder(nn.Module):
  def __init__(self, latent_size):
    super(Decoder,self).__init__()

    self.decoder = nn.Sequential(
                      nn.Linear(latent_size, 16),
                      nn.ReLU(),
                      nn.Linear(16,32),
                      nn.ReLU(),
                      nn.Linear(32,67),
                      nn.Sigmoid())

  def forward(self,x):
    return self.decoder(x)

In [ ]:
encoder = Encoder(latent_size).to(device)
decoder = Decoder(latent_size).to(device)
criterion = torch.nn.MSELoss()
optimizer = torch.optim.Adam(list(encoder.parameters())+list(decoder.parameters()), lr = 0.001)

In [ ]:
for ep in range(300):

  running_loss = 0.0

  for images, label in train_loader:
    inputs = images.to(device)
    optimizer.zero_grad()
    latent_var = encoder(inputs)
    outputs = decoder(latent_var)
    loss = criterion(inputs, outputs)
    loss.backward()
    optimizer.step()
    running_loss+=loss.item()
  cost = running_loss/len(train_loader)
  print('[%d]loss:%.3f'%(ep,cost))
  torch.save(encoder.state_dict(), './models/en_pth')
  torch.save(decoder.state_dict(), './models/de_pth')

[0]loss:0.234
[1]loss:0.227
[2]loss:0.219
[3]loss:0.207
[4]loss:0.190
[5]loss:0.164
[6]loss:0.126
[7]loss:0.091
[8]loss:0.078
[9]loss:0.077
[10]loss:0.075
[11]loss:0.073
[12]loss:0.073
[13]loss:0.073
[14]loss:0.073
[15]loss:0.073
[16]loss:0.073
[17]loss:0.074
[18]loss:0.073
[19]loss:0.073
[20]loss:0.072
[21]loss:0.073
[22]loss:0.073
[23]loss:0.073
[24]loss:0.072
[25]loss:0.073
[26]loss:0.072
[27]loss:0.072
[28]loss:0.072
[29]loss:0.073
[30]loss:0.073
[31]loss:0.072
[32]loss:0.073
[33]loss:0.073
[34]loss:0.072
[35]loss:0.072
[36]loss:0.072
[37]loss:0.074
[38]loss:0.072
[39]loss:0.073
[40]loss:0.072
[41]loss:0.073
[42]loss:0.072
[43]loss:0.072
[44]loss:0.072
[45]loss:0.073
[46]loss:0.072
[47]loss:0.072
[48]loss:0.072
[49]loss:0.071
[50]loss:0.072
[51]loss:0.072
[52]loss:0.072
[53]loss:0.071
[54]loss:0.072
[55]loss:0.071
[56]loss:0.071
[57]loss:0.070
[58]loss:0.070
[59]loss:0.069
[60]loss:0.068
[61]loss:0.068
[62]loss:0.068
[63]loss:0.066
[64]loss:0.066
[65]loss:0.066
[66]loss:0.066
[67]l

In [ ]:
#forward training data
encoder.load_state_dict(torch.load('/content/models/en_pth'))
decoder.load_state_dict(torch.load('/content/models/de_pth'))

#latent vector
latent_features_trn = []
actual_trn = []

with torch.no_grad():
  for images, labels in train_loader:
    inputs = images.to(device)
    labels = labels.to(device)
    latent_var = encoder(inputs)

    latent_features_trn+=latent_var.cpu().tolist()
    actual_trn+=labels.cpu().tolist()

In [ ]:
#forward test data
encoder.load_state_dict(torch.load('/content/models/en_pth'))
decoder.load_state_dict(torch.load('/content/models/de_pth'))

#latent vector
latent_features_tes = []
actual_tes = []

with torch.no_grad():
  for images, labels in test_loader:
    inputs = images.to(device)
    labels = labels.to(device)
    latent_var = encoder(inputs)

    latent_features_tes+=latent_var.cpu().tolist()
    actual_tes+=labels.cpu().tolist()

In [ ]:
latent_features_trn = np.array(latent_features_trn)
latent_features_tes = np.array(latent_features_tes)
actual_tes = np.array(actual_tes)
actual_trn = np.array(actual_trn)

In [ ]:
latent_features_trn=pd.DataFrame(latent_features_trn)
latent_features_tes=pd.DataFrame(latent_features_tes)
actual_tes = pd.DataFrame(actual_tes)
actual_trn = pd.DataFrame(actual_trn)

In [ ]:
latent_features_trn.to_csv('/content/drive/MyDrive/지역 치안 안전 데이터 분석/data/Latent Variable/대전 cluster 17/latent_features_trn.csv')
latent_features_tes.to_csv('/content/drive/MyDrive/지역 치안 안전 데이터 분석/data/Latent Variable/대전 cluster 17/latent_features_tes.csv')
actual_tes.to_csv('/content/drive/MyDrive/지역 치안 안전 데이터 분석/data/Latent Variable/대전 cluster 17/actual_tes.csv')
actual_trn.to_csv('/content/drive/MyDrive/지역 치안 안전 데이터 분석/data/Latent Variable/대전 cluster 17/actual_trn.csv')

In [ ]:
actual_trn = actual_trn.values.reshape(-1,)
actual_tes = actual_tes.values.reshape(-1,)

In [ ]:
clf = RandomForestClassifier(n_estimators = 500, criterion = 'entropy', random_state = 0)
clf.fit(latent_features_trn, actual_trn)
y_trn_pred = clf.predict(latent_features_trn)

print(classification_report(actual_trn,y_trn_pred))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00       546
           1       1.00      1.00      1.00       355

    accuracy                           1.00       901
   macro avg       1.00      1.00      1.00       901
weighted avg       1.00      1.00      1.00       901



In [ ]:
y_pred = clf.predict(latent_features_tes)
print(classification_report(actual_tes,y_pred))

              precision    recall  f1-score   support

           0       0.67      0.77      0.71       137
           1       0.54      0.42      0.47        89

    accuracy                           0.63       226
   macro avg       0.60      0.59      0.59       226
weighted avg       0.62      0.63      0.62       226

